IMPORT


In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import math


from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.api import OLS
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000


#path = '/Koding_With_Kolesh/challenges/RMB_NOWCAST_APRIL/'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_Mar_23.csv')
vehicles = pd.read_csv('Naamsa_Vehicle_Sales.csv')
seed = 3

PIVOTING THE DATA


In [36]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

ADD THE april ROW


In [37]:
date_str = '2023-04-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING


In [38]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,9):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)

HANDLE MISSING DATA


In [39]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION


In [40]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-04-30"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-04-30"]

training_set = train[train['Month']!= '2023-03-31']
validation_set = train[train['Month']== '2023-03-31']

train.shape, test.shape, training_set.shape, validation_set.shape

((14, 118), (1, 118), (13, 118), (1, 118))

MODELING


In [41]:
import optuna


target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}
bp = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-5, 100)  # Ridge regularization strength
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    copy_X = trial.suggest_categorical('copy_X', [True, False])
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])

    r_model = Ridge(alpha=alpha, fit_intercept=fit_intercept, copy_X=copy_X, max_iter=max_iter, tol=tol, solver=solver,)
    r_model.fit(X_train, y_train[target_col])
    y_pred_colr = r_model.predict(X_val)
    return mean_squared_error(y_val[target_col], y_pred_colr)

#training
for target_col in target_cols:
    #study = optuna.create_study(direction='minimize')
    #study.optimize(objective, n_trials=50)
    #best_params = study.best_params
    
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.1, random_state=seed)
    el_model = ElasticNet(alpha=0.1, l1_ratio=0.5, warm_start=True, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    #bp[target_col] = best_params
#validation
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)

#scoring

y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T

df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})
dfx = pd.DataFrame({'y_pred': y_predx.flatten(), 'y_val': y_val.values.flatten()})
dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})
dfr = pd.DataFrame({'y_pred': y_predr.flatten(), 'y_val': y_val.values.flatten()})
dfel = pd.DataFrame({'y_pred': y_predel.flatten(), 'y_val': y_val.values.flatten()})

#calculate the rmse
rmse = np.sqrt(mean_squared_error(df['y_pred'], df['y_val']))
rmsex = np.sqrt(mean_squared_error(dfx['y_pred'], df['y_val']))
rmsel = np.sqrt(mean_squared_error(dfl['y_pred'], df['y_val']))
rmser = np.sqrt(mean_squared_error(dfr['y_pred'], df['y_val']))
rmseel = np.sqrt(mean_squared_error(dfel['y_pred'], df['y_val']))

print(f'RMSE of Liner Regression: {rmse}') 
print(f'RMSE of XGB: {rmsex}') 
print(f'RMSE of Lasso Regression: {rmsel}') 
print(f'RMSE of Riged Regression: {rmser}') 
print(f'RMSE of Elastic Regression: {rmseel}') 

RMSE of Liner Regression: 0.8381601857653865
RMSE of XGB: 1.9937501699261877
RMSE of Lasso Regression: 1.7700467504576742
RMSE of Riged Regression: 1.3699553316157576
RMSE of Elastic Regression: 1.6648872111925421


In [42]:
#best_params

In [43]:
X_train = train[features]
y_train = train[target_cols]

X_val = test[features]
y_val = test[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.1, random_state=seed)
    el_model = ElasticNet(alpha=0.1, l1_ratio=0.5, warm_start=True, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    
#pridiction
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)
    
y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T

print(f'pridiction of Liner Regression: {y_pred}') 
print(f'prediction of XGB: {y_predx}') 
print(f'pridiction of Riged Regression: {y_predr}') 
print(f'prediction of Lasso Regression: {y_predl}') 
print(f'pridiction of Elastic Net Regression: {y_predel}') 

pridiction of Liner Regression: [[109.59796062 103.83856228  99.60706275 109.04790209 118.02513998
  109.79847297 110.01172476 108.17981097 104.57304738 109.46013532
  104.65879206 109.85706087 114.29335271]]
prediction of XGB: [[109.198746 103.4979    99.37331  110.39879  116.265495 108.99878
  109.09883  107.79874  104.49803  107.89879  104.19867  108.63675
  113.915596]]
pridiction of Riged Regression: [[109.67784767 103.52422465  99.4490024  109.894742   117.17349331
  109.44792753 109.19629183 108.12586573 104.33693376 108.59374572
  104.59961041 109.53350689 114.32640035]]
prediction of Lasso Regression: [[109.00593156 102.84658564  99.68571429 108.91096486 116.58238675
  108.33258924 108.61428191 106.82194519 103.84       108.02867963
  103.23981601 108.79222921 115.84325648]]
pridiction of Elastic Net Regression: [[109.57191199 103.21071214  99.59748044 108.91189941 117.65426528
  109.20377824 108.75135288 107.66561356 104.29635857 108.23864009
  103.72642553 109.40374384 115.4

SUBMITION


In [44]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']
    
    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)
    
    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')
    
    return sub_df[['ID', 'Value']]

In [45]:
sub = prepSub(y_predl, target_cols, test, 'April') 
sub.to_csv('out/lasso_with_lag8_historic_check.csv', index=False)